In [1]:
from dask.distributed import Client, progress

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=40)
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://10.32.16.243:35419
distributed.scheduler - INFO -   dashboard at:                     :8787


In [2]:
## path for mdules
import sys
sys.path.insert(0,"/home/jovyan/xscale")
import xscale


sys.path.insert(0,"/home/jovyan/eNATL60-plots-paper/common-lib/")
import GriddedData

## imports

import numpy as np
import numpy.ma as ma
import xarray as xr
import time

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.cm as mplcm
seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

### quick plot
import matplotlib.pyplot as plt

import glob
import os 

%matplotlib inline

In [3]:
import gcsfs
fs = gcsfs.GCSFileSystem('pangeo-181919', requester_pays=True)


In [4]:
zmap_grid = fs.get_mapper('pangeo-meom/eNATL60-I/eNATL60-mesh-mask')
ds_grid = xr.open_zarr(zmap_grid)


distributed.scheduler - INFO - Register tcp://10.32.10.5:40051
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.10.5:40051
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.43.27:36495
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.43.27:36495
distributed.core - INFO - Starting established connection


In [5]:
zmap_u = fs.get_mapper('pangeo-meom/eNATL60-BLBT02-SSU-1h')
ds_u = xr.open_zarr(zmap_u)


In [6]:
zmap_v = fs.get_mapper('pangeo-meom/eNATL60-BLBT02-SSV-1h')
ds_v = xr.open_zarr(zmap_v)


In [7]:
navlat= ds_grid['nav_lat']
navlon= ds_grid['nav_lon']
e1u=ds_grid.e1u[0]
e1v=ds_grid.e1v[0]
e2u=ds_grid.e2u[0]
e2v=ds_grid.e2v[0]
ff=ds_grid.ff[0]


In [8]:
def filt(w):
    win_box2D = w.window
    win_box2D.set(window='hanning', cutoff=90, dim=['x', 'y'], n=[90, 90])
    bw = win_box2D.boundary_weights(drop_dims=[])
    w_LS = win_box2D.convolve(weights=bw)
    w_SS=w-w_LS
    return w_SS


In [9]:
def curl(u,v,e1v,e2u,ff):
    '''
    This routine computes the relative vorticity from 2D fields of horizontal velocities and the spatial Coriolis parameter.
    '''
    #Computation of dy(u)
    fe2u=1/e2u
    fse2u=fe2u.squeeze()
    dyu=(u.shift(y=-1) - u)*fse2u
    #Computation of dx(v)
    fe1v=1/e1v
    fse1v=fe1v.squeeze()
    dxv=(v.shift(x=-1) - v)*fse1v
    #Projection on the grid T
    dxvt=0.5*(dxv.shift(y=-1)+dxv)
    dyut=0.5*(dyu.shift(x=-1)+dyu)
    #Computation of the vorticity divided by f
    fff=1/ff
    curl=(dxvt-dyut)*fff
    return curl

In [10]:
def plot_fine_scale_variance(var,loncrs,latcrs,lon,lat,hpvarm,month,m,config,case):
    ''' map of the averaged fine scale variance
    '''
    fig, ax = plt.subplots(1,1,figsize=(15,10))
    ax = plt.subplot(111,projection=ccrs.PlateCarree(central_longitude=0))
    ax.autoscale(tight=True)
    cont=np.isnan(hpvarm)
    gl = ax.gridlines(draw_labels=True, linestyle=':', color='black',
                      alpha=0.5)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 25, 'color': 'gray'}
    gl.ylabel_style = {'size': 25, 'color': 'gray'}
    
    ax.tick_params('both',labelsize=25)

    pcolor = ax.pcolormesh(loncrs,latcrs,ma.masked_invalid(var),cmap=seq_cmap,vmin=0,vmax=0.1,alpha=1)
    ax.contour(lon,lat,cont,alpha=0.5,linewidth=0.000001,antialiased=True,colors='black')
    cbar = plt.colorbar(pcolor,orientation='horizontal',pad=0.1)
    cbar.ax.tick_params(labelsize=25)
    cbar.set_label('Small scales surface vorticity variance in '+month+' for '+config+'-'+case,fontsize=15)
    plt.savefig(config+'-'+case+'_m'+m+'_fine_scale_variance_vorticity.png')
  

In [11]:
    print('Select dates in zarr')
    u=ds_u.sel(time_counter=slice('2010-03-01','2010-03-31'))['sozocrtx']
    v=ds_v.sel(time_counter=slice('2010-03-01','2010-03-31'))['somecrty']


Select dates in zarr


In [12]:
print(100*100*2000*u[0].dtype.itemsize/ (1000**2))

80.0


In [13]:
u_rechunk=u.chunk({'time_counter':10,'y':4000,'x':5000})
v_rechunk=v.chunk({'time_counter':10,'y':4000,'x':5000})

In [ ]:
    print('Computing curl')
    curl_surf=curl(u_rechunk,v_rechunk,e1v,e2u,ff)
    print('Filtering curl')
    curl_SS=filt(curl_surf)
    curl_LS=curl_surf-curl_SS
    hpcurl=curl_SS
    hpcurl2 = hpcurl ** 2
    hpcurl2m = hpcurl2.mean(axis=0,keep_attrs=True)
    navlat2=np.array(navlat).squeeze()
    navlon2=np.array(navlon).squeeze()
    print('Coarsening of the grid')
    mgrd = GriddedData.grid2D(navlat=navlat2, navlon=navlon2)
    crs = GriddedData.grdCoarsener(mgrd,crs_factor=60)
    hpcurl2mc = crs.return_ravel(np.asarray(hpcurl2m))
    hpcurl2mcm = np.mean(hpcurl2mc,axis=-3)
    latcrs=crs.return_ravel(np.asarray(navlat2))
    loncrs=crs.return_ravel(np.asarray(navlon2))
    latcrsm=np.mean(latcrs,axis=-3)
    loncrsm=np.mean(loncrs,axis=-3)
      

Computing curl
Filtering curl
Coarsening of the grid


distributed.scheduler - INFO - Register tcp://10.32.24.3:45003
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.24.3:45003
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.24.4:39369
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.24.4:39369
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.28.3:33521
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.28.3:33521
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.28.2:43857
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.28.2:43857
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.34.3:33601
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.34.3:33601
distributed.core - INFO

In [ ]:
print('Plotting')
plot_fine_scale_variance(hpcurl2mcm,loncrsm, latcrsm,navlon,navlat,hpcurl2m,'March',month,config,case)


In [ ]:
print('Select dates in zarr')
u=ds_u.sel(time_counter=slice('2009-09-01','2009-09-30'))['sozocrtx']
v=ds_v.sel(time_counter=slice('2009-09-01','2009-09-30'))['somecrty']
u_rechunk=u.chunk({'time_counter':1,'y':4729,'x':8354})
v_rechunk=v.chunk({'time_counter':1,'y':4729,'x':8354})
print('Computing curl')
curl_surf=curl(u_rechunk,v_rechunk,e1v,e2u,ff)
print('Filtering curl')
curl_SS=filt(curl_surf)
curl_LS=curl_surf-curl_SS
hpcurl=curl_SS
hpcurl2 = hpcurl ** 2
hpcurl2m = hpcurl2.mean(axis=0,keep_attrs=True)
navlat2=np.array(navlat).squeeze()
navlon2=np.array(navlon).squeeze()
print('Coarsening of the grid')
mgrd = GriddedData.grid2D(navlat=navlat2, navlon=navlon2)
crs = GriddedData.grdCoarsener(mgrd,crs_factor=60)
hpcurl2mc = crs.return_ravel(np.asarray(hpcurl2m))
hpcurl2mcm = np.mean(hpcurl2mc,axis=-3)
latcrs=crs.return_ravel(np.asarray(navlat2))
loncrs=crs.return_ravel(np.asarray(navlon2))
latcrsm=np.mean(latcrs,axis=-3)
loncrsm=np.mean(loncrs,axis=-3)
print('Plotting')
plot_fine_scale_variance(hpcurl2mcm,loncrsm, latcrsm,navlon,navlat,hpcurl2m,'September',month,config,case)
      